In [ ]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T # Transpose the dev set
Y_dev = data_dev[0]
X_dev = data_dev[1:n] # X_dev will now have 784 rows (features) and 1000 columns (samples)
X_dev = X_dev / 255.0 # Normalize pixel values

data_train  = data[1000:m].T # Transpose the training set
Y_train = data_train[0]
X_train = data_train[1:n] # X_train will have 784 rows and (m-1000) columns
X_train = X_train / 255.0 # Normalize pixel values


In [ ]:
def init_params():
    W1 = np.random.randn(10, 784) * 0.01 
    B1 = np.zeros((10, 1)) 
    W2 = np.random.randn(10, 10) * 0.01
    B2 = np.zeros((10, 1))
    return W1, B1, W2, B2

def ReLU(Z):
    return np.maximum(0, Z)

def softMax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True)) 
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)
    
def forward_prop(W1, B1, W2, B2, X):
    Z1 = W1.dot(X)+B1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1)+B2
    A2 = softMax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    Y = np.array(Y) # Create a one-hot encoded matrix with shape (number of classes, number of samples)
    one_hot_Y = np.zeros((Y.max() + 1, Y.size)) # Set the appropriate elements to 1
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = X.shape[1] # Number of samples
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True) 
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

def updatee_params(W1, B1, W2, B2,  dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha  * dW1
    B1 = B1 - alpha * db1 
    W2 = W2 - alpha * dW2
    B2 = B2 - alpha * db2 
    return W1, B1, W2, B2

In [ ]:
def get_predictions(A2):
    predictions = np.argmax(A2, axis=0)
    return predictions

def get_accuracy(predictions, Y):
    Y = np.array(Y)
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size
    

def gradient(X, Y, iterations, alpha):
    W1, B1, W2, B2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, B1, W2, B2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, B1, W2, B2 = updatee_params(W1, B1, W2, B2, dW1, db1, dW2, db2, alpha)
        if i % 500 == 0:
            print(f"Iteration: {i}") 
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f"Accuracy: {accuracy:.4f}") 
            
            
    return W1, B1, W2, B2 
    

In [ ]:
W1, B1, W2, B2 = gradient(X_train, Y_train, 3000, 0.01)

In [ ]:
def make_predictions (X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions (A2)
    return predictions

def test_prediction (index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions (X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction:", prediction)
    print("Label: ", label)
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
dev_predictions = make_predictions(X_dev, W1, B1, W2, B2)
get_accuracy(dev_predictions, Y_dev)

In [ ]:
test_prediction(2, W1, B1, W2, B2)  # Test with the image in the training set